## Лабораторная работа №6

Задание 1 . Провести классификацию найденного датасета, методами CatBoost . В формате Markdown написать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

In [1]:
!pip install catboost

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from catboost import CatBoostClassifier
from imblearn.under_sampling import RandomUnderSampler
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('classified_data.csv')
data

,ID,Loan Amount,Funded Amount,Funded Amount Investor,Term,Batch Enrolled,Interest Rate,Grade,Sub Grade,Employment Duration,...,Recoveries,Collection Recovery Fee,Collection 12 months Medical,Application Type,Last week Pay,Accounts Delinquent,Total Collection Amount,Total Current Balance,Total Revolving Credit Limit,Loan Status
0,65087372,10000,32236,12329.36286,59,BAT2522922,11.135007,B,C4,MORTGAGE,...,2.498291,0.793724,0,INDIVIDUAL,49,0,31,311301,6619,0
1,1450153,3609,11940,12191.99692,59,BAT1586599,12.237563,C,D3,RENT,...,2.377215,0.974821,0,INDIVIDUAL,109,0,53,182610,20885,0
2,1969101,28276,9311,21603.22455,59,BAT2136391,12.545884,F,D4,MORTGAGE,...,4.316277,1.020075,0,INDIVIDUAL,66,0,34,89801,26155,0
3,6651430,11170,6954,17877.15585,59,BAT2428731,16.731201,C,C3,MORTGAGE,...,0.107020,0.749971,0,INDIVIDUAL,39,0,40,9189,60214,0
4,14354669,16890,13226,13539.92667,59,BAT5341619,15.008300,C,D4,MORTGAGE,...,1294.818751,0.368953,0,INDIVIDUAL,18,0,430,126029,22579,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67458,16164945,13601,6848,13175.28583,59,BAT3193689,9.408858,C,A4,MORTGAGE,...,564.614852,0.865230,0,INDIVIDUAL,69,0,48,181775,34301,1
67459,35182714,8323,11046,15637.46301,59,BAT1780517,9.972104,C,B3,RENT,...,2.015494,1.403368,0,INDIVIDUAL,14,0,37,22692,8714,0
67460,16435904,15897,32921,12329.45775,59,BAT1761981,19.650943,A,F3,MORTGAGE,...,5.673092,1.607093,0,INDIVIDUAL,137,0,17,176857,42330,0
67461,5300325,16567,4975,21353.68465,59,BAT2333412,13.169095,D,E3,OWN,...,1.157454,0.207608,0,INDIVIDUAL,73,0,61,361339,39075,0


In [3]:
label_encoder = LabelEncoder()

data['Grade Encoded'] = label_encoder.fit_transform(data['Grade'])
data['Sub Grade Encoded'] = label_encoder.fit_transform(data['Sub Grade'])
data['Employment Duration Encoded'] = label_encoder.fit_transform(data['Employment Duration'])
data['Verification Status Encoded'] = label_encoder.fit_transform(data['Verification Status'])
data['Loan Title Encoded'] = label_encoder.fit_transform(data['Loan Title'])
data['Initial List Status Encoded'] = label_encoder.fit_transform(data['Initial List Status'])

non_numeric_columns = data.select_dtypes(exclude=['number']).columns.tolist()
data = data.drop("ID", axis=1)
data = data.drop("Accounts Delinquent", axis=1)

data = data.drop(columns=non_numeric_columns)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67463 entries, 0 to 67462
Data columns (total 30 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Loan Amount                   67463 non-null  int64  
 1   Funded Amount                 67463 non-null  int64  
 2   Funded Amount Investor        67463 non-null  float64
 3   Term                          67463 non-null  int64  
 4   Interest Rate                 67463 non-null  float64
 5   Home Ownership                67463 non-null  float64
 6   Debit to Income               67463 non-null  float64
 7   Delinquency - two years       67463 non-null  int64  
 8   Inquires - six months         67463 non-null  int64  
 9   Open Account                  67463 non-null  int64  
 10  Public Record                 67463 non-null  int64  
 11  Revolving Balance             67463 non-null  int64  
 12  Revolving Utilities           67463 non-null  float64
 13  T

In [6]:
X = data.drop(columns=['Loan Status'])
y = data['Loan Status']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
under_sampler = RandomUnderSampler(random_state=42)

X_resampled, y_resampled = under_sampler.fit_resample(X_train, y_train)

catboost_params = {
    'iterations': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'depth': [4, 6, 8]
}

catboost_model = CatBoostClassifier(verbose=False)

catboost_grid_search = GridSearchCV(catboost_model, catboost_params, cv=5)
catboost_grid_search.fit(X_resampled, y_resampled)

best_catboost_params = catboost_grid_search.best_params_
print(f"Лучшие параметры для CatBoost: {best_catboost_params}")

Лучшие параметры для CatBoost: {'depth': 4, 'iterations': 300, 'learning_rate': 0.1}


In [7]:
best_catboost_model = CatBoostClassifier(**best_catboost_params, verbose=False)
best_catboost_model.fit(X_resampled, y_resampled)

y_pred_catboost = best_catboost_model.predict(X_test)

accuracy_catboost = accuracy_score(y_test, y_pred_catboost)
print(f"Точность CatBoost на тестовых данных: {accuracy_catboost}")

print("Classification Report для CatBoost:")
print(classification_report(y_test, y_pred_catboost))

Точность CatBoost на тестовых данных: 0.5314985918276595
Classification Report для CatBoost:
              precision    recall  f1-score   support

           0       0.91      0.54      0.68     18386
           1       0.10      0.48      0.16      1853

    accuracy                           0.53     20239
   macro avg       0.50      0.51      0.42     20239
weighted avg       0.84      0.53      0.63     20239

